In [ ]:
from flask import Flask, request, jsonify, render_template, redirect, url_for, session, flash
import sqlite3
import hashlib

app = Flask(__name__)
app.secret_key = '874546574574576'  # Change this to a secure, random key in production

# Function to hash passwords
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Function to check password
def check_password(stored_password, provided_password):
    return stored_password == hash_password(provided_password)

# API endpoint for user registration
@app.route('/api/register', methods=['POST'])
def api_register():
    data = request.json

    if not data or 'email' not in data or 'password' not in data:
        return jsonify({"error": "Email and password are required"}), 400

    email = data['email']
    password = hash_password(data['password'])

    try:
        # Connect to the database
        conn = sqlite3.connect('users.db')
        c = conn.cursor()

        # Insert new user into the database
        c.execute("INSERT INTO users (email, password) VALUES (?, ?)", (email, password))
        conn.commit()
        conn.close()

        return jsonify({"message": "User registered successfully"}), 201

    except sqlite3.IntegrityError:
        return jsonify({"error": "Email already exists"}), 400

# API endpoint for user login
@app.route('/api/login', methods=['POST'])
def api_login():
    data = request.json

    if not data or 'email' not in data or 'password' not in data:
        return jsonify({"error": "Email and password are required"}), 400

    email = data['email']
    password = data['password']

    # Connect to the database
    conn = sqlite3.connect('users.db')
    c = conn.cursor()

    # Query the database for the user
    c.execute('SELECT password FROM users WHERE email = ?', (email,))
    user = c.fetchone()
    conn.close()

    if user and check_password(user[0], password):
        session['email'] = email
        return jsonify({"message": "Login successful"}), 200
    else:
        return jsonify({"error": "Invalid email or password"}), 401


@app.route('/')
def home():
    return render_template('home.html')


# Logout route
@app.route('/logout')
def logout():
    session.pop('email', None)
    return redirect(url_for('home'))

# Login form route
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        # Connect to the database
        conn = sqlite3.connect('users.db')
        c = conn.cursor()

        # Query the database for the user
        c.execute('SELECT password FROM users WHERE email = ?', (email,))
        user = c.fetchone()
        conn.close()

        if user and check_password(user[0], password):
            session['email'] = email
            return redirect(url_for('home'))
        else:
            flash('Invalid email or password')

    return render_template('login.html')

# Registration form route
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        email = request.form['email']
        password = hash_password(request.form['password'])

        try:
            # Connect to the database
            conn = sqlite3.connect('users.db')
            c = conn.cursor()

            # Insert new user into the database
            c.execute("INSERT INTO users (email, password) VALUES (?, ?)", (email, password))
            conn.commit()
            conn.close()

            return redirect(url_for('login'))

        except sqlite3.IntegrityError:
            flash('Email already exists')

    return render_template('register.html')

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Aug/2024 20:41:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 20:41:41] "GET /static/sick1.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/2024 20:41:41] "GET /static/tips4.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/2024 20:41:41] "GET /static/about1.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/2024 20:41:41] "GET /static/plant4.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/2024 20:42:14] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 20:42:25] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 20:42:36] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 20:44:28] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 20:44:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 20:44:29] "GET /static/sick1.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/2024 20:44:29] "GET /static/tips4.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/2024 20:44:29] "GET /static/plant4.png HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/20